[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruliana/pytorch-katas/blob/main/dan_1/kata_02_temple_door_humidity_predictor_unrevised.ipynb)

## 🏮 The Ancient Scroll Unfurls 🏮

**COOK OH-PAI-TIMIZER'S DOOR WISDOM: THE SIGMOID GATE MYSTERY**

Dan Level: 1 (Temple Sweeper) | Time: 45 minutes | Sacred Arts: Binary Classification, Sigmoid Activation, Threshold Decisions

## 📜 THE CHALLENGE

Cook Oh-Pai-Timizer bustles through the temple corridors, balancing steaming bowls of sacred soup and muttering about the ancient wooden doors. "These old doors," Cook says, wiping sweat from their brow, "they have minds of their own! When the humidity rises, some stick like they're guarded by stubborn spirits, while others swing freely. Yesterday I nearly spilled an entire pot of precious lotus root broth trying to push through the meditation hall door!"

The wise cook has been observing patterns for months, noting how the temple's humidity affects each door's behavior. Now Cook seeks to master the art of prediction—to know which doors will stick before approaching them with precious cargo. "If I can learn this pattern," Cook explains, "I can plan my routes and avoid the sticky doors entirely. But I need more than just intuition—I need the mathematical wisdom of the sigmoid function to transform humidity measurements into clear yes-or-no decisions!"

## 🎯 THE SACRED OBJECTIVES

By the end of this kata, you will have mastered:

- [ ] **Binary Classification Fundamentals**: Learn to predict yes/no outcomes using neural networks
- [ ] **Sigmoid Activation Mastery**: Transform continuous values into probabilities between 0 and 1
- [ ] **Single-Variable Classification**: Build intuition with one input feature before tackling complex problems
- [ ] **Threshold Decision Making**: Convert probabilities into actionable binary decisions
- [ ] **Binary Cross-Entropy Loss**: Understand the mathematics of classification error measurement
- [ ] **Probability Interpretation**: Learn to read and trust your model's confidence levels

In [ ]:
# 📦 ALL IMPORTS AND CONFIGURATION
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

# Global configuration constants
DEFAULT_CHAOS_LEVEL = 0.1
HUMIDITY_THRESHOLD = 60.0  # Cook's observed critical humidity level

print("🏮 The Temple of Neural Networks welcomes you, Grasshopper!")
print(f"PyTorch version: {torch.__version__}")
print("🐱 Suki stirs from her afternoon nap, sensing the approach of learning...")

## 🍜 THE SACRED DATA GENERATION SCROLL

In [ ]:
def generate_door_humidity_data(n_observations: int = 200, chaos_level: float = 0.1) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate observations of temple door behavior based on humidity levels.
    
    Cook Oh-Pai-Timizer's wisdom: Doors tend to stick when humidity > 60%
    But ancient wood has its own mysterious patterns!
    
    Args:
        n_observations: Number of door-testing incidents to simulate
        chaos_level: Amount of wooden unpredictability (0.0 = perfectly predictable doors, 1.0 = chaos)
        
    Returns:
        Tuple of (humidity_levels, door_sticks) as sacred tensors
        door_sticks: 1 = door sticks (avoid!), 0 = door opens smoothly
    """
    # Temple humidity ranges from 20% (dry winter) to 90% (monsoon season)
    humidity_levels = torch.rand(n_observations, 1) * 70 + 20
    
    # Cook's observed pattern: doors stick more often when humidity > 60%
    # Create base probability using a smooth sigmoid-like pattern
    stick_probability = torch.sigmoid((humidity_levels.squeeze() - HUMIDITY_THRESHOLD) / 5.0)
    
    # Add wooden chaos - sometimes doors surprise you!
    chaos = torch.randn(n_observations) * chaos_level * 0.3
    stick_probability = torch.clamp(stick_probability + chaos, 0.0, 1.0)
    
    # Convert probabilities to actual door behavior (0 or 1)
    door_sticks = torch.bernoulli(stick_probability).unsqueeze(1)
    
    return humidity_levels, door_sticks

def visualize_door_wisdom(humidity: torch.Tensor, door_sticks: torch.Tensor, 
                         predictions: torch.Tensor = None):
    """
    Display the sacred patterns of door behavior vs humidity.
    """
    plt.figure(figsize=(14, 8))
    
    # Create two subplots
    plt.subplot(1, 2, 1)
    
    # Separate sticky and smooth doors for clear visualization
    sticky_mask = door_sticks.squeeze() == 1
    smooth_mask = door_sticks.squeeze() == 0
    
    plt.scatter(humidity[sticky_mask].numpy(), [1]*torch.sum(sticky_mask).item(), 
                alpha=0.6, color='red', s=50, label='Doors That Stick (Danger!)')
    plt.scatter(humidity[smooth_mask].numpy(), [0]*torch.sum(smooth_mask).item(), 
                alpha=0.6, color='green', s=50, label='Doors That Open Smoothly')
    
    plt.axvline(x=HUMIDITY_THRESHOLD, color='orange', linestyle='--', alpha=0.7,
                label=f'Cook\'s Threshold ({HUMIDITY_THRESHOLD}% humidity)')
    
    plt.xlabel('Humidity Level (%)')
    plt.ylabel('Door Behavior')
    plt.title('Cook Oh-Pai-Timizer\'s Door Observations')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(-0.2, 1.2)
    plt.yticks([0, 1], ['Opens Smoothly', 'Sticks!'])
    
    # Second subplot for predictions if provided
    if predictions is not None:
        plt.subplot(1, 2, 2)
        
        # Sort by humidity for smooth prediction curve
        sorted_indices = torch.argsort(humidity.squeeze())
        sorted_humidity = humidity[sorted_indices]
        sorted_predictions = predictions[sorted_indices]
        
        plt.plot(sorted_humidity.numpy(), sorted_predictions.detach().numpy(), 
                'gold', linewidth=3, label='Your Sigmoid Predictions')
        plt.scatter(humidity[sticky_mask].numpy(), [1]*torch.sum(sticky_mask).item(), 
                    alpha=0.4, color='red', s=30)
        plt.scatter(humidity[smooth_mask].numpy(), [0]*torch.sum(smooth_mask).item(), 
                    alpha=0.4, color='green', s=30)
        
        plt.axhline(y=0.5, color='purple', linestyle=':', alpha=0.7,
                    label='Decision Threshold (50%)')
        plt.axvline(x=HUMIDITY_THRESHOLD, color='orange', linestyle='--', alpha=0.7)
        
        plt.xlabel('Humidity Level (%)')
        plt.ylabel('Predicted Probability of Sticking')
        plt.title('Your Mystical Door Predictions')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.ylim(-0.1, 1.1)
    
    plt.tight_layout()
    plt.show()

# Generate the sacred data
humidity_levels, door_sticks = generate_door_humidity_data(n_observations=200)

print(f"🚪 Generated {len(humidity_levels)} observations of temple door behavior")
print(f"💧 Humidity range: {humidity_levels.min():.1f}% to {humidity_levels.max():.1f}%")
print(f"🔒 Doors that stick: {torch.sum(door_sticks).item()}/{len(door_sticks)} ({torch.mean(door_sticks).item():.1%})")

# Visualize the sacred patterns
visualize_door_wisdom(humidity_levels, door_sticks)

## 🚪 THE SIGMOID GATEWAY PREDICTOR

In [ ]:
class DoorStickinessPredictor(nn.Module):
    """A mystical artifact for predicting when temple doors will misbehave."""
    
    def __init__(self, input_features: int = 1):
        super(DoorStickinessPredictor, self).__init__()
        # TODO: Create a Linear layer to transform humidity into raw predictions
        # Hint: One input (humidity), one output (raw stickiness score)
        self.linear = None
        
        # TODO: Add the sigmoid activation function
        # Hint: torch.nn.Sigmoid() transforms any number into a probability (0 to 1)
        self.sigmoid = None
    
    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """Transform humidity measurements into door-sticking probabilities."""
        # TODO: Pass humidity through the linear layer first
        raw_output = None
        
        # TODO: Apply sigmoid to convert raw output to probability (0-1 range)
        # This is the magic that makes classification work!
        probability = None
        
        return probability

def train_door_predictor(model: nn.Module, features: torch.Tensor, target: torch.Tensor,
                        epochs: int = 2000, learning_rate: float = 0.1) -> list:
    """
    Train the door stickiness prediction model.
    
    Returns:
        List of loss values during training
    """
    # TODO: Choose the right loss function for binary classification
    # Hint: Binary Cross Entropy Loss is the master's choice for yes/no problems
    criterion = None
    
    # TODO: Choose your optimizer
    # Hint: SGD with higher learning rate works well for simple problems
    optimizer = None
    
    losses = []
    
    for epoch in range(epochs):
        # TODO: CRITICAL - Clear gradients from previous iteration
        # The spirits of old gradients must be banished!
        
        # TODO: Forward pass - get probability predictions
        predictions = None
        
        # TODO: Calculate binary classification loss
        loss = None
        
        # TODO: Backward pass - compute gradients
        
        # TODO: Update model parameters
        
        losses.append(loss.item())
        
        # Report progress to Cook Oh-Pai-Timizer
        if (epoch + 1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
            if loss.item() < 0.3:
                print("🍜 Cook Oh-Pai-Timizer nods approvingly - the wisdom flows!")
    
    return losses

def make_door_decisions(model: nn.Module, humidity: torch.Tensor, threshold: float = 0.5) -> torch.Tensor:
    """
    Convert probability predictions into binary door decisions.
    
    Args:
        model: Your trained predictor
        humidity: Humidity measurements
        threshold: Decision boundary (default 0.5 means 50% confidence)
    
    Returns:
        Binary decisions: 1 = door will stick (avoid!), 0 = door opens smoothly
    """
    with torch.no_grad():
        probabilities = model(humidity)
        decisions = (probabilities > threshold).float()
    return decisions

## ⚡ THE TRIALS OF MASTERY

### Trial 1: Basic Sigmoid Mastery
- [ ] Loss decreases smoothly (no oscillating spirits)
- [ ] Final loss below 0.4 (Cook Oh-Pai-Timizer's approval threshold)
- [ ] Model outputs probabilities between 0 and 1 (sigmoid magic working)
- [ ] Predictions show clear S-curve pattern when plotted against humidity

### Trial 2: Decision Accuracy Test
- [ ] Achieve >75% accuracy on training data
- [ ] Model correctly identifies most doors above 60% humidity as "sticky"
- [ ] Model correctly identifies most doors below 60% humidity as "smooth"

### Trial 3: Understanding Test

In [ ]:
def test_your_wisdom(model):
    """Cook Oh-Pai-Timizer's evaluation of your door prediction skills."""
    
    # Test sigmoid activation is working
    test_humidity = torch.tensor([[30.0], [60.0], [80.0]])  # Low, medium, high humidity
    predictions = model(test_humidity)
    
    # All predictions should be probabilities (0 to 1)
    assert torch.all(predictions >= 0) and torch.all(predictions <= 1), \
        "Sigmoid not working - predictions outside 0-1 range!"
    
    # Higher humidity should generally mean higher sticking probability
    assert predictions[2] > predictions[0], \
        "High humidity should be stickier than low humidity!"
    
    # Check model learned reasonable threshold behavior
    low_humidity_pred = model(torch.tensor([[40.0]])).item()
    high_humidity_pred = model(torch.tensor([[75.0]])).item()
    
    assert low_humidity_pred < 0.5, f"Low humidity prediction {low_humidity_pred:.3f} should be < 0.5"
    assert high_humidity_pred > 0.5, f"High humidity prediction {high_humidity_pred:.3f} should be > 0.5"
    
    print("🎉 Cook Oh-Pai-Timizer beams with pride!")
    print("   'You have mastered the sigmoid way - doors shall no longer surprise you!'")
    print(f"   Low humidity (40%): {low_humidity_pred:.1%} chance of sticking")
    print(f"   High humidity (75%): {high_humidity_pred:.1%} chance of sticking")

## 🌸 THE FOUR PATHS OF MASTERY: PROGRESSIVE EXTENSIONS

*Master Pai-Torch gestures toward four different pathways leading deeper into the temple.*

"You have learned the fundamental way, grasshopper. But mastery comes through exploring the branching paths."

🔍 **Adjust the confidence threshold.** What happens when you use 0.3 or 0.7 instead of 0.5 for making decisions? How does this affect accuracy?

⚡ **Increase or decrease the learning rate.** What happens to the sigmoid curve shape? Can you make the training converge faster or more stable?

🎯 **Add more chaos to the door data.** How much randomness can your model handle before it starts making poor predictions?

🌟 **Change the humidity threshold in the data generation.** What if doors start sticking at 50% or 80% humidity instead of 60%? Does your model adapt?

## 🏆 COMPLETION CEREMONY

*Cook Oh-Pai-Timizer approaches with a ceremonial ladle*

"Congratulations, young grasshopper! You have successfully mastered the Sigmoid Gateway Mystery! Through the ancient temple doors, you have learned the fundamental mysteries of binary classification:

**Sacred Knowledge Acquired:**
- **Binary Classification Mastery**: You can now predict yes/no outcomes with neural networks
- **Sigmoid Activation Understanding**: You transform raw outputs into meaningful probabilities
- **Decision Threshold Wisdom**: You know how to convert predictions into actionable decisions
- **Binary Cross-Entropy Fluency**: You measure classification errors with the proper loss function
- **Confidence Interpretation**: You understand what probability predictions really mean

**Final Wisdom:**
Remember always: the sigmoid function is like the temple gate - it stands between chaos and order, transforming any input into the sacred realm of probability. This wisdom will serve you well in your journey toward greater mastery!

🐱 *Suki purrs approvingly from her perch, as if to say: "You are ready for greater challenges, young neural warrior."*

🏮 **May your gradients flow smoothly and your losses converge swiftly!** 🏮